In [ ]:
# https://huggingface.co/unum-cloud/uform-gen/tree/main
!huggingface-cli download unum-cloud/uform-gen --cache-dir "C:\shared-drive\llm_models\uform-gen"


In [4]:
# need torch for cuda version 12.1 since that's what I have installed
#!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121
# !pip3 install torch==2.1.1+cu121 torchvision==0.17.2+cu121 torchaudio==2.1.1 --extra-index-url https://download.pytorch.org/whl/cu121

!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.3.0%2Bcu121-cp310-cp310-win_amd64.whl (2413.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0.dev20240527+cu121
    Can't uninstall 'torch'. No files were found to uninstall.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import torch 
print(torch.cuda.is_available())
print(torch.cuda.device_count())
!python --version
print(f"PyTorch version: {torch.__version__}")

# Print the CUDA version
print(f"CUDA version: {torch.version.cuda}")

num_devices = torch.cuda.device_count()
for device_id in range(num_devices):
    device_name = torch.cuda.get_device_name(device_id)
    print(f"Device ID: {device_id}, Name: {device_name}")

True
2
Python 3.10.6
PyTorch version: 2.3.0+cu121
CUDA version: 12.1
Device ID: 0, Name: NVIDIA GeForce RTX 3090
Device ID: 1, Name: NVIDIA GeForce GTX 1050 Ti


In [7]:
!pip show flash_attn

In [6]:
#!pip install uform
# !pip install torchvision
!pip install safetensors


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import torch
from PIL import Image
from uform.gen_model import VLMForCausalLM, VLMProcessor
from transformers import AutoModel, AutoProcessor
device_id = 0 # 1 runs out of memory
torch.cuda.set_device(device_id)

# Set the device
device = torch.device(f'cuda:{device_id}' if torch.cuda.is_available() else 'cpu')
print(f'device is {device}')
model = AutoModel.from_pretrained("unum-cloud/uform-gen2-qwen-500m", trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("unum-cloud/uform-gen2-qwen-500m", trust_remote_code=True)
# 
# prompt = "Describe this picture"
# image = Image.open("panda.png")
# 
# inputs = processor(text=[prompt], images=[image], return_tensors="pt")
# with torch.inference_mode():
#      output = model.generate(
#         **inputs,
#         do_sample=False,
#         use_cache=True,
#         max_new_tokens=256,
#         eos_token_id=151645,
#         pad_token_id=processor.tokenizer.pad_token_id
#     )
# 
# prompt_len = inputs["input_ids"].shape[1]
# decoded_text = processor.batch_decode(output[:, prompt_len:])[0]
# 
# print(decoded_text)


device is cuda:0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
def analyze_image(image_path:str, prompt:str):
    image = Image.open(image_path)
    
    inputs = processor(text=[prompt], images=[image], return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.inference_mode():
         output = model.generate(
            **inputs,
            do_sample=False,
            use_cache=True,
            max_new_tokens=256,
            eos_token_id=151645,
            pad_token_id=processor.tokenizer.pad_token_id
        )
    
    prompt_len = inputs["input_ids"].shape[1]
    decoded_text = processor.batch_decode(output[:, prompt_len:])[0]
    
    # print(decoded_text)
    return decoded_text

In [33]:
prompt = '''You are an expert in providing in depth analysis of images. 
You provide nuanced details of the objects, lighting, colors, actions, time, place, feeling, smells, and other details.
Please describe the image in as much detail as possible. Identify any people, places, or things in the image. 
Describe the facial expressions and emotions of any people present.

List and describe all objects in the scene, including details such as color, shape, size, texture, and feel.

Specify the time of day depicted in the image and provide an approximate year or historical period, if possible.

Identify the country, city, building, or specific location where the image is set.

Indicate whether the image is a painting, picture, drawing, or another type of visual representation.

Describe the overall vibe or emotion that the image conveys.

Detail any actions taking place in the image.

Transcribe any visible text in the image.

Mention any other notable features or elements that contribute to the overall understanding of the image.

'''
analyze_image("panda.png", prompt)

"The image depicts a playful and whimsical scene featuring a giant panda bear wearing an astronaut outfit. The panda bear is seated at a wooden table, surrounded by various objects. The table is adorned with a plate of noodles, a cup of coffee, and a pair of chopsticks. The panda bear is wearing an astronaut uniform, complete with an American flag on its back. The astronaut uniform is white with blue accents, and the panda bear's outfit is made of a material that resembles a space suit. The panda bear's eyes are closed, and its ears are perked up, giving it a curious and playful expression. The panda bear's face is black and white, and its ears are black as well. The table is made of wood and has a brown color. The background consists of a wall with white tiles, and there is a window with blinds. The overall atmosphere of the image is light-hearted and fun, with the panda bear appearing to be enjoying a meal in a unique and imaginative setting.<|im_end|>"

In [34]:
import torch
from PIL import Image
from transformers import StoppingCriteria, StoppingCriteriaList

def analyze_image_stream(image_path: str, prompt: str):
    image = Image.open(image_path)
    
    inputs = processor(text=[prompt], images=[image], return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    class StreamStoppingCriteria(StoppingCriteria):
        def __init__(self, prompt_length):
            self.prompt_length = prompt_length
            self.generated_text = ""
        
        def __call__(self, input_ids, scores, **kwargs):
            new_token_id = input_ids[0, -1].item()
            new_token_text = processor.tokenizer.decode([new_token_id])
            self.generated_text += new_token_text
            print(new_token_text, end='', flush=True)  # Print without newline and flush immediately
            # You can add custom stopping conditions here if needed
            return False

    stopping_criteria = StoppingCriteriaList([StreamStoppingCriteria(inputs["input_ids"].shape[1])])
    
    with torch.inference_mode():
        output = model.generate(
            **inputs,
            do_sample=False,
            use_cache=True,
            max_new_tokens=1024,
            eos_token_id=151645,
            pad_token_id=processor.tokenizer.pad_token_id,
            stopping_criteria=stopping_criteria
        )

    prompt_len = inputs["input_ids"].shape[1]
    decoded_text = stopping_criteria[0].generated_text
    
    # return decoded_text

In [36]:
prompt = '''You are an expert in providing in depth analysis of images. 
You provide nuanced details of the subjects, objects, lighting, colors, actions, time, place, feeling, smells, and other details. i.e. You should provide details on who, what, why, when, where.
Please provide an in depth analysis of the image.
'''
analyze_image_stream("panda.png", prompt)

The image features a playful scene of a panda bear wearing an astronaut suit, sitting at a table. The panda bear is quite large, occupying a significant portion of the image. It has a black and white coat, with a distinct black nose and ears. The panda bear is wearing a white astronaut suit, which includes an American flag on the back. The suit is also adorned with a blue and white stripe, and the bear's arms are adorned with a pair of black and white paws.
The panda bear is seated on a brown wooden chair, which is positioned in front of a table. The table is filled with various objects. On the table, there is a plate with noodles, which the panda bear is eating. There is also a cup with a straw in it, and a pair of chopsticks placed on the table.
The background of the image features a wall with white tiles. The wall is partially visible, and it appears to be a part of a larger room. The wall has a brown color, and there is a window with a white frame. The window is located behind the 

In [54]:
prompt = '''You are an expert in providing in depth analysis of images. 
You provide nuanced details of the subjects, objects, lighting, colors, actions, time, place, feeling, smells, and other details. i.e. You should provide details on who, what, why, when, where.
Please provide an in depth analysis of the image.
'''
analyze_image_stream("C:\\shared-drive\\llm_models\\ComfyUI_windows_portable\\ComfyUI\\output\\ComfyUI_00192_.png", prompt)

The image captures a young woman standing in a park. The woman is the central figure, occupying a significant portion of the image. She is dressed in a green top and a white skirt, both of which are part of a pair. The top is a bright green, and the skirt is a contrasting white. The woman's hair is long and dark, and she is wearing a necklace around her neck. Her face is turned towards the camera, giving a direct gaze. Her left hand is on her hip, and her right hand is in her pocket.
The park is lush and green, with trees scattered around. The trees are tall and leafy, and their trunks are visible. The ground is covered in grass, and there are patches of dirt visible as well. The woman is standing on a path that is surrounded by the trees. The path is well-maintained, and it seems to be a quiet and peaceful area.
The lighting in the image is bright and natural, with sunlight filtering through the trees. The sunlight illuminates the woman's face and the background, creating a warm and i

In [64]:
prompt = '''
You are an expert in providing in depth analysis of images. 
You provide nuanced details of the subjects, objects, lighting, colors, actions, time, place, feeling, smells, and other details. i.e. You should provide details on who, what, why, when, where.
Please provide an in depth analysis of the image.
'''
analyze_image_stream("Drivers_ License_1.jpg", prompt)

The image is a 2012 Pennsylvania State University football credit card. The card is in a light blue color and features a photo of a young man, who is smiling. The man is wearing a green shirt and has short brown hair. The card has the university's logo and the date "09/01/2012" written on it. The card also has the university's phone number and the university's website address. The card is in good condition and is clearly labeled with the credit card number, university name, and date of issuance.<|im_end|>